<a href="https://colab.research.google.com/github/AndreyV80/NetStudy/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22API_ProjectWork%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Задание 1.**
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from re import search

In [57]:
res = requests.get('https://habr.com/ru/all/')
res

<Response [200]>

Поскольку на текущей странице искомых слов не было, добавил для проверки имеющиеся

In [58]:
KEYWORDS = ['python', 'парсинг', 'alfa Battle Camp', 'Visualize']


In [59]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [60]:
posts = soup.find_all('article', class_='post')
# posts

In [ ]:
# добираемся до блоков с новостями
news_blocks = soup.find_all('h2', class_='post__title')
news_blocks
# print(news_blocks.text)

In [62]:

news_blocks2 = soup.find_all('article',class_= 'post__time') 
##, class_="post__time")
type(news_blocks2)
news_blocks2

[]

Согласно лекции нашел в каких классах имеется необходимая информация. Циклом вытащил исходную информацию текущей страницы. Согласно комментарию преподавателя добавил условие для поиска. Сложность в обновлении информации, на момент кода искомых слов не было.

In [63]:
url_habr = []       #список ссылок
text_habr = []      #список заголовков
time_habr = []
for test in posts:
    for a in test.find_all('a', class_='post__title_link'):
        #print(a.text)
        for k in KEYWORDS:
           # print(k)
            if search(k.lower(),a.text.lower()): #преобразуем к одному регистру
                text_habr.append(a.text)
                url_habr.append(a.get('href'))
                for link2 in test.find_all('span', class_='post__time'):
                     time_habr.append(link2.text)          
date = {'time_habr' : time_habr,'text_habr' : text_habr, 'news': url_habr}
date_pd = pd.DataFrame(date)
date_pd

,time_habr,text_habr,news
0,вчера в 18:40,Топ 6 библиотек Python для визуализации: какую...,https://habr.com/ru/company/otus/blog/558478/


**Задание 2.**
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [2]:
emails2 = {'emailAddresses':['xxx@x.ru', 'yyy@y.com']}

In [3]:
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

In [ ]:
HEADERS2 = {
    'Vaar-Version':'0', 'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36', 'emailAddresses': 'xxx@x.ru'}

In [4]:
HEADERS3 = {
    'Vaar-Version': '0',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Header-App-Build-Version': '1.0.0',
}

In [5]:
reg2 = requests.post(URL, json=emails2, headers=HEADERS3)

In [ ]:
reg2

<Response [200]>

In [6]:
print(reg2.json())

{'breaches': {'17110': {'breachId': 17110, 'site': 'azcentral.com', 'recordsCount': 705538, 'description': 'At an unconfirmed date, online Arizona newspaper "AZ Central" was allegedly breached. The stolen data contains passwords and email addresses. This breach is being publicly shared on the internet.', 'publishDate': '2020-01-03T00:00:00Z', 'statistics': {'usernames': 0, 'passwords': 702971, 'emails': 705538}}, '17670': {'breachId': 17670, 'site': 'wishbone.io', 'recordsCount': 37475476, 'description': 'In January 2020, the online poll website Wishbone was allegedly breached. The stolen data contains usernames, passwords, email addresses and additional personal information. This breach is being privately shared on the internet.', 'publishDate': '2020-05-28T00:00:00Z', 'statistics': {'usernames': 31111127, 'passwords': 10648734, 'emails': 10234249}}, '13094': {'breachId': 13094, 'site': 'myheritage.com', 'recordsCount': 110041653, 'description': 'In October 2017, a customer database b

In [10]:
regj = reg2.json()

In [21]:
type(regj)

dict

In [ ]:
regj['breaches']

description - описание; 
publishDate - дата публикации; 
site - место утечки

In [43]:
list_dict= []
for breaches in regj['breaches'].values():
    
    list_dict.append(breaches)

In [45]:
data = pd.DataFrame(list_dict)

In [50]:
data.head(1)

,breachId,site,recordsCount,description,publishDate,statistics
0,17110,azcentral.com,705538,"At an unconfirmed date, online Arizona newspap...",2020-01-03T00:00:00Z,"{'usernames': 0, 'passwords': 702971, 'emails'..."


In [51]:
dataRes = data[['publishDate', 'site', 'description']]

In [55]:
dataRes.rename(columns = {'publishDate':'Date_of_leak', 'site':'leak_location'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [56]:
dataRes.head()

,Date_of_leak,leak_location,description
0,2020-01-03T00:00:00Z,azcentral.com,"At an unconfirmed date, online Arizona newspap..."
1,2020-05-28T00:00:00Z,wishbone.io,"In January 2020, the online poll website Wishb..."
2,2017-11-04T00:00:00Z,myheritage.com,"In October 2017, a customer database belonging..."
3,2021-02-11T00:00:00Z,forums.vkmonline.com,"At an unconfirmed date, the Russian-language m..."
4,2019-06-13T00:00:00Z,canva.com,"In May 2019, graphic-design site Canva's datab..."
